In [92]:
import pandas as pd
from glob import glob
import os
from typing import List, Set
from statistics import mean

In [26]:
ground_truth = pd.read_csv('merged_annotation.csv', header=None, names=('Path', 'Score'), sep=',', skipinitialspace=True, quotechar="'")
ground_truth['id'] = ground_truth['Path'].str[:-4]
ground_truth.set_index('id', inplace=True)
# ground_truth.loc['55-If3ebb17bc7963bfa8d81f040'].Score (=='Off Topic')

In [108]:
def calculate_precision_at_k(run_result_path : str, k : int):
    df = pd.read_csv(run_result_path, delim_whitespace=True, header=None, names=('QueryId', 'Stance', 'ImageId', 'Score', 'Rank', 'Tag'))
    
    precision_history = []
    for (image_id, stance), results in df.groupby(['QueryId', 'Stance']):
        true_positives = 0
        for _, row in results.iterrows():
            if row['Rank'] > k:
                continue

            lookup_id = f'{row["QueryId"]}-{row["ImageId"]}'
            score = ground_truth.loc[lookup_id].Score
            if (stance == 'pro' and score == 'Pro') or (stance == 'con' and score == 'Con'):
                true_positives += 1
        precision_history.append(true_positives/k)
    
    print(f'Precision at {k}: {mean(precision_history)}')

prec = calculate_precision_at_k('run-results/run3.txt', 3)

Precision at 3: 0.09999999999999999


    QueryId Stance                    ImageId  Score  Rank Tag
10       51    con  If0312920d8e2b14e67471275      1    10   -
11       51    con  I65be4ac0c003a24940c977be      2     9   -
12       51    con  Ib5922f507114277155dc96e9      3     8   -
13       51    con  I8afb105ac66beff412a54681      4     7   -
14       51    con  Ia9eee868fd1fbb2fdcdf41ea      5     6   -
15       51    con  I5223eaa67229e284b12f9414      6     5   -
16       51    con  Ic5dbf002236fdadbcd432f98      7     4   -
17       51    con  Ia146f5c11acc7bdcb2621d8b      8     3   -
18       51    con  I77664a737969ae4b2c429b8f      9     2   -
19       51    con  I98eb652a8f63ba6f71858848     10     1   -
